In [1]:
from model import *
from data import *
import cv2

Using TensorFlow backend.


In [2]:
model_checkpoint = ModelCheckpoint('unet_weights.hdf5', monitor='loss',verbose=1, save_best_only=True)
model = unet()

## Train dengan data Generator

In [ ]:
data_gen_args = dict()#rotation_range=0.2,
                    #width_shift_range=0.05,
                    #height_shift_range=0.05,
                    #shear_range=0.05,
                    #zoom_range=0.05,
                    #horizontal_flip=True,
                    #fill_mode='nearest')
myGene = trainGenerator(2,'data/train','image','label',data_gen_args,save_to_dir = None)
model.fit_generator(myGene, steps_per_epoch = 2000, epochs = 5, callbacks = [model_checkpoint])

## Train dengan npy file

In [3]:
imgs_train, imgs_mask_train = geneTrainNpy("data/train/aug/","data/train/aug/")
#model.fit(imgs_train, imgs_mask_train, batch_size = 2, epochs = 10, verbose = 1, validation_split= 0.2, shuffle = True, callbacks = [model_checkpoint])
model.fit(imgs_train, imgs_mask_train, batch_size = 2, epochs = 100, verbose = 1, shuffle = True, validation_split= 0.2, callbacks = [model_checkpoint])

0 [==============================] - 525s 656ms/step - loss: 0.0723 - accuracy: 0.9803 - val_loss: 0.0154 - val_accuracy: 0.9942

Epoch 00001: loss improved from inf to 0.07226, saving model to unet_weights.hdf5
Epoch 2/100
800/800 [==============================] - 509s 637ms/step - loss: 0.0186 - accuracy: 0.9942 - val_loss: 0.0154 - val_accuracy: 0.9967

Epoch 00002: loss improved from 0.07226 to 0.01859, saving model to unet_weights.hdf5
Epoch 3/100
800/800 [==============================] - 492s 615ms/step - loss: 0.0094 - accuracy: 0.9970 - val_loss: 0.0043 - val_accuracy: 0.9986

Epoch 00003: loss improved from 0.01859 to 0.00940, saving model to unet_weights.hdf5
Epoch 4/100
800/800 [==============================] - 514s 643ms/step - loss: 0.0113 - accuracy: 0.9963 - val_loss: 0.0072 - val_accuracy: 0.9978

Epoch 00004: loss did not improve from 0.00940
Epoch 5/100
800/800 [==============================] - 524s 655ms/step - loss: 0.0068 - accuracy: 0.9976 - val_loss: 0.0058 -

## Simpan Data Prediksi

In [3]:
testGene = testGenerator('data/test')
results = cv2.normalize(src=model.predict_generator(testGene,100,verbose=1), dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
saveResult("data/test", results)

100/100 [==============================] - 31s 307ms/step
